In [1]:
from corrective_rag import corrective_rag_translated, corrective_rag
from simple_agent_rag import simple_agent_rag, simple_agent_rag_translated
from naive_rag import naive_rag_translated, naive_rag
from react_rag import react_rag, react_rag_translated
from init_vectorstore import init_vectorstore, init_semantic_vectorstore
from evaluate_model import evaluate_model
from ensemble_model import ensemble_models
from semantic_model import semantic_model, semantic_model_translated
from ragas_func import ragas_with_params
import os
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from datasets import load_dataset
import matplotlib.pyplot as plt
os.environ["OPENAI_API_KEY"] = "sk-proj-94aoBcsuvI9IT4SwqEo1T3BlbkFJ9e3g3snpnEk3MFKn4e4X"

## Naive rag returner hele konteksten (hele konteksten er svaret)
## simple agent returner hele konteksten (hele konteksten er svaret)
## semantic model returnerer to lister: svar og kontekst
## react returnerer to lister: svar og kontekst
## corrective RAG returnerer to lister: svar og kontekst

In [2]:
instances = 10
file_path = "/Users/adrianfolge/Documents/lokal:skole/Master/data/synthetic_data/vol2_questions_and_answers_ytterligere_revidert.csv"

In [3]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [4]:
databases = init_vectorstore(embeddings, text_splitter)
databases_translated = init_vectorstore(embeddings, text_splitter, translate=True)
semantic_databases = init_semantic_vectorstore()

100%|██████████| 10/10 [00:09<00:00,  1.06it/s]
Created a chunk of size 1147, which is longer than the specified 500
Created a chunk of size 1570, which is longer than the specified 500
Created a chunk of size 639, which is longer than the specified 500
Created a chunk of size 610, which is longer than the specified 500
Created a chunk of size 1008, which is longer than the specified 500
Created a chunk of size 545, which is longer than the specified 500
Created a chunk of size 536, which is longer than the specified 500
Created a chunk of size 583, which is longer than the specified 500
Created a chunk of size 1162, which is longer than the specified 500
Created a chunk of size 607, which is longer than the specified 500
Created a chunk of size 1206, which is longer than the specified 500
Created a chunk of size 697, which is longer than the specified 500
Created a chunk of size 734, which is longer than the specified 500
Created a chunk of size 916, which is longer than the specified

In [5]:
references = load_dataset('csv', data_files=file_path, split=f"train[:{instances}]")
refs = references["svar"]
questions = references["spørsmål"]

In [6]:
corrective_rag_translated_list, corrective_rag_translated_context = corrective_rag_translated(instances, file_path, databases_translated)
simple_agent_rag_list = simple_agent_rag(instances, file_path, databases)
simple_agent_rag_translated_list = simple_agent_rag_translated(instances, file_path, databases_translated)
naive_rag_list = naive_rag(instances, file_path, databases)
naive_rag_translated_list = naive_rag_translated(instances, file_path, databases_translated)
react_rag_translated_list,react_rag_translated_context  = react_rag_translated(instances, file_path, databases_translated)
react_rag_list, react_rag_context = react_rag(instances, file_path, databases)
semantic_rag_list, semantic_rag_context = semantic_model(instances, file_path, semantic_databases)

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK RELEVANCE---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
"Node 'grade_documents':"
'\n---\n'
---DECIDE TO GENERATE---
{'documents': [Document(page_content='Municipal sub-plan for the city centre\n\nPlan provisions\n\nAdopted by the city council on 26 August 2021\n\n1\n\n1.\n\nMAIN INTENTIONS OF THE PLAN ................................................. ................................................ .............. 3\n\n2.\n\nLEGAL EFFECT OF THE PLAN ................................................ ................................................ ...................... 3\n\n3.', metadata={'source': '../data/pdfs/Kristiansund.pdf'}), Document(page_content="The new opera and cultural center must be integrated into city life • It is desirable to have more housing projects in the city centre, where children's living environment is also taken care of • The mai

In [7]:
simple_agent_rag_score = ragas_with_params(simple_agent_rag_list, questions, simple_agent_rag_list, refs)
corrective_rag_translated__score = ragas_with_params(simple_agent_rag_list, questions, simple_agent_rag_list, refs)
simple_agent_translated_rag_score = ragas_with_params(simple_agent_rag_translated_list, questions, simple_agent_rag_translated_list, refs)
naive_rag_score = ragas_with_params(naive_rag_list, questions, naive_rag_list, refs)
naive_rag_translated_score = ragas_with_params(naive_rag_translated_list, questions, naive_rag_translated_list, refs)
react_rag_translated_score = ragas_with_params(react_rag_translated_list, questions, react_rag_translated_context, refs)
react_rag_score = ragas_with_params(react_rag_list, questions, react_rag_context, refs)
semantic_rag_score = ragas_with_params(semantic_rag_list, questions, semantic_rag_context, refs)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.


In [8]:
print(simple_agent_rag_score) 
print(corrective_rag_translated__score) 
print(simple_agent_translated_rag_score)
print(naive_rag_score) 
print(naive_rag_translated_score) 
print(react_rag_translated_score) 
print(react_rag_score)
print(semantic_rag_score) 

{'context_precision': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.9602, 'context_recall': 0.7857}
{'context_precision': 1.0000, 'faithfulness': 0.9889, 'answer_relevancy': 0.9566, 'context_recall': 0.7857}
{'context_precision': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.9714, 'context_recall': 0.7222}
{'context_precision': 0.9000, 'faithfulness': 1.0000, 'answer_relevancy': 0.8127, 'context_recall': 0.8500}
{'context_precision': 1.0000, 'faithfulness': 0.9750, 'answer_relevancy': 0.8236, 'context_recall': 1.0000}
{'context_precision': 1.0000, 'faithfulness': 0.7738, 'answer_relevancy': 0.9070, 'context_recall': 0.9500}
{'context_precision': 1.0000, 'faithfulness': 0.6250, 'answer_relevancy': 0.9342, 'context_recall': 0.9750}
{'context_precision': 0.7000, 'faithfulness': 0.8750, 'answer_relevancy': 0.7586, 'context_recall': 0.8750}


In [ ]:
references = load_dataset('csv', data_files=file_path, split=f"train[:{instances}]")
refs = references["svar"]
print("Scores for corrective RAG")
corrective_rag_score = evaluate_model(corrective_rag_translated_list, refs, instances)
print("#############")
print("Scores for simple agent")
simple_agent_score = evaluate_model(simple_agent_rag_list, refs, instances)
print("#############")
print("Scores for simple agent translated")
simple_agent_translated_score = evaluate_model(simple_agent_rag_translated_list, refs, instances)
print("#############")
print("Scores for naive rag")
naive_rag_score = evaluate_model(naive_rag_list, refs, instances)
print("#############")
print("Scores for naive rag translated")
naive_rag_translated_score = evaluate_model(naive_rag_translated_list, refs, instances)
print("#############")
print("Scores for react RAG")
react_rag_score = evaluate_model(react_rag_list, refs, instances)
print("#############")
print("Scores for react RAG translated")
react_rag_translated_score = evaluate_model(react_rag_translated_list, refs, instances)
print("#############")

In [ ]:
def create_predictions_dict(*lists):
    predictions_dict = {}
    for i, lst in enumerate(lists, start=1):
        key = f"Prediction {i}"
        predictions_dict[key] = lst
    return predictions_dict

In [ ]:
predictions_dict = create_predictions_dict(corrective_rag_translated_list, simple_agent_rag_list, simple_agent_rag_translated_list, naive_rag_list, naive_rag_translated_list, react_rag_translated_list, react_rag_list)
ensembling_models = ensemble_models(predictions_dict, references, instances, list_of_context)
print("Scores for the ensembled model")
ensemble_models_score = evaluate_model(ensembling_models, refs, instances)
print("#############")

In [ ]:
# Scores for each model (example data)
model_names = ["Corrective RAG", "Simple agent", "Simple agent translated", "Naive rag", "Naive rag translated", "React rag", "React rag translated", "Model ensembling"]
scores = [corrective_rag_score, simple_agent_score, simple_agent_translated_score, naive_rag_score, naive_rag_translated_score, react_rag_score, react_rag_translated_score, ensemble_models_score]

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(model_names, scores, color='skyblue')
plt.xlabel('Models')
plt.ylabel('Scores')
plt.title('Scores of Different Models')
plt.ylim(0, 1)  # Set y-axis limits to 0 and 1
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45, ha='right')
plt.show()